In [ ]:
import os
import numpy as np
import sys
import holoviews as hv

sys.path.append('/Users/annateruel/phd_code/')
from ca2img.minian_dca1 import (preprocessing,  
                                start_cluster,
                                motion_correction,
                                initialize, 
                                
                                run_cnmf)

from minian.utilities import (open_minian,
                              get_optimal_chk,
                              save_minian)
from minian.motion_correction import apply_transform, estimate_motion
from minian.visualization import visualize_motion

In [ ]:
video_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'

In [ ]:
hv.notebook_extension("bokeh", width=100)

# Minian Analysis in dCA1

The parameters used here for minian pipeline are the ones we chose for our videos from dCA1 recordings

In [ ]:
start_cluster(n_workers=8, memory_limit="6GB")

## Preprocessing

In [ ]:
int_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/23-01-ST-anna/AD22-118/annateruel/srt_trial/AD22-118/minian/2023_01_24/12_41_16/My_V4_Miniscope/'
video_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/23-01-ST-anna/AD22-118/annateruel/srt_trial/AD22-118/minian/2023_01_24/12_41_16/My_V4_Miniscope/'
preprocessing(intpath=int_dir, dpath=video_dir, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 7}, 
                          param_background_removal={"method": "tophat", "wnd": 15})

In [ ]:
dirs = ['/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-113']

for dir in dirs:
    for root, _, _ in os.walk(dir):
        if root.endswith("My_V4_Miniscope"):
            preprocessing(intpath=root, dpath=root, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 5}, 
                          param_background_removal={"method": "tophat", "wnd": 5})

## Motion correction

In [ ]:
param_estimate_motion = {"dim": "frame"}

param_save_minian = {
    "dpath": video_dir,  
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}

minian_ds = open_minian(int_dir)
varr_ref = minian_ds['varr_ref']
chk, _ = get_optimal_chk(varr_ref, dtype=float)
motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

In [ ]:
root_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-113'

param_estimate_motion = {"dim": "frame"}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        minian_ds = open_minian(int_dir)
        varr_ref = minian_ds['varr_ref']
        chk, _ = get_optimal_chk(varr_ref, dtype=float)

        param_save_minian = {
            "dpath": int_dir,  
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        %time 
        motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

In [ ]:
visualize_motion(motion)

In [ ]:
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
os.environ["MINIAN_INTERMEDIATE"] = int_dir

param_save_minian = {
    "dpath": int_dir,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}
param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8} 

varr_ref = open_minian(int_dir)['varr_ref']
chk = get_optimal_chk(varr_ref, dtype=float)

Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

initialize(Y_fm_chk, 
           Y_hw_chk, 
           param_save_minian, 
           param_seeds_init, 
           param_pnr_refine, 
           param_ks_refine, 
           param_seeds_merge, 
           param_initialize, 
           param_init_merge, 
           int_dir, 
           chk)

In [ ]:
print("A data sample:", A.isel(unit_id=0).values)
print("C data sample:", C.isel(unit_id=0).values)

In [ ]:
print("Y_fm_chk data sample:", Y_fm_chk.isel(frame=0, height=0, width=0).values)
print("C_chk data sample:", C_chk.isel(frame=0, unit_id=0).values)

In [ ]:
from holoviews.operation.datashader import regrid
from holoviews.operation.datashader import regrid, datashade
from holoviews.operation.datashader import regrid, datashade

output_size = 100
hv.output(size=int(output_size * 0.55))
im_opts = dict(
    frame_width=500,
    aspect=A.sizes["width"] / A.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
cr_opts = dict(frame_width=750, aspect=1.5 * A.sizes["width"] / A.sizes["height"])
(
    regrid(
        hv.Image(
            A.max("unit_id").rename("A").compute().astype(np.float32),
            kdims=["width", "height"],
        ).opts(**im_opts)
    ).relabel("Initial Spatial Footprints")
    + regrid(
        hv.Image(
            C.rename("C").compute().astype(np.float32), kdims=["frame", "unit_id"]
        ).opts(cmap="viridis", colorbar=True, **cr_opts)
    ).relabel("Initial Temporal Components")
).cols(2)

In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-115'

param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        param_save_minian = {
            "dpath": int_dir,
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        varr_ref = open_minian(int_dir)['varr_ref']
        chk = get_optimal_chk(varr_ref, dtype=float)

        Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
        Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

        initialize(Y_fm_chk, 
                   Y_hw_chk, 
                   param_save_minian, 
                   param_seeds_init, 
                   param_pnr_refine, 
                   param_ks_refine, 
                   param_seeds_merge, 
                   param_initialize, 
                   param_init_merge, 
                   int_dir, 
                   chk)

In [ ]:
print("A data sample:", A.isel(unit_id=0).values)
print("C data sample:", C.isel(unit_id=0).values)

In [ ]:
print("Y_fm_chk data sample:", Y_fm_chk.isel(frame=0, height=0, width=0).values)
print("C_chk data sample:", C_chk.isel(frame=0, unit_id=0).values)

In [ ]:
from holoviews.operation.datashader import regrid
from holoviews.operation.datashader import regrid, datashade
from holoviews.operation.datashader import regrid, datashade

output_size = 100
hv.output(size=int(output_size * 0.55))
im_opts = dict(
    frame_width=500,
    aspect=A.sizes["width"] / A.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
cr_opts = dict(frame_width=750, aspect=1.5 * A.sizes["width"] / A.sizes["height"])
(
    regrid(
        hv.Image(
            A.max("unit_id").rename("A").compute().astype(np.float32),
            kdims=["width", "height"],
        ).opts(**im_opts)
    ).relabel("Initial Spatial Footprints")
    + regrid(
        hv.Image(
            C.rename("C").compute().astype(np.float32), kdims=["frame", "unit_id"]
        ).opts(cmap="viridis", colorbar=True, **cr_opts)
    ).relabel("Initial Temporal Components")
).cols(2)

In [5]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-120/annateruel/social-discrimination-test/AD23-120/customEntValHere/2024_03_01/15_42_34'
param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.02, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 5, "thres_corr": 0.8, "noise_freq": 0.02}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.02}
param_init_merge = {"thres_corr": 0.8}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        param_save_minian = {
            "dpath": int_dir,
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        varr_ref = open_minian(int_dir)['varr_ref']
        chk = get_optimal_chk(varr_ref, dtype=float)

        Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
        Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

        initialize(Y_fm_chk, 
                   Y_hw_chk, 
                   param_save_minian, 
                   param_seeds_init, 
                   param_pnr_refine, 
                   param_ks_refine, 
                   param_seeds_merge, 
                   param_initialize, 
                   param_init_merge, 
                   int_dir, 
                   chk)

In [ ]:
int_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_16/15_53_12/My_V4_Miniscope/'

minian_ds = open_minian(int_dir)
A = minian_ds['A']
A_init = minian_ds['A_init']


In [ ]:
has_nans = np.isnan(A_init).any().compute()
all_zeros = np.all(A_init == 0).compute()

print(f"Array A has NaNs: {has_nans}")
print(f"All values in array A are zeros: {all_zeros}")

In [ ]:

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        minian_ds = open_minian(int_dir)
        for var_name in ['A', 'C']:
            var = minian_ds[var_name]
            has_nans = np.isnan(var).any().compute()
            all_zeros = np.all(var == 0).compute()

            print(f"Array {var_name} has NaNs: {has_nans}")
            print(f"All values in array {var_name} are zeros: {all_zeros}")

## CNMF

Let's briefly explain the parameters we have defined in the following cell, in order to set them properly

1. `param_get_noise = {"noise_range": (0.06, 0.5)}` specifies the range of frequencies to be considered as noise. Lower frequencies generally correspond to signal, while higher frequencies correspond to noise. 

2. `param_first_spatial = {"dl_wnd": 10,"sparse_penal": 0.01,"size_thres": (25, None)}`: 
-  `dl_wnd` is the size of the window used to smooth the data spatially
- `sparse_penal` is sparsity penalty for spatial component, promoting a more sparse representation, if you decrease the sparsity penalty you will allow for denser spatial component
- `size_threshold` is the size of the spatial components to be considered

3. `param_first_temporal = {"noise_freq": 0.06,"sparse_penal": 1,"p": 1,"add_lag": 20,"jac_thres": 0.2}`
- `noise_freq` frequency  below which the signal is considered noise. If you lower the noise frequency you will include more signal components. 
- `sparse_penal` sparsity penalty for temporal component. If you decrese the spartisy penalty you will allow more temporal fluctuations
- `add_lag` is the number of additional lags to consider in the temporal model
- `jac_thres` threshold for the jacobian determinant, used to filter out low-quality components. If you decrease the filter you will retain more components. 

These are the main parameters to consider


In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/23-01-ST-anna/AD22-118/annateruel/srt_trial/AD22-118/customEntValHere/'

param_get_noise = {"noise_range": (0.06, 0.5)}
param_first_spatial = {
    "dl_wnd": 10,
    "sparse_penal": 0.01,
    "size_thres": (5, None)}
param_first_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1.5,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.2}
param_first_merge = {"thres_corr": 0.8}
param_second_spatial = {
    "dl_wnd": 10,
    "sparse_penal": 0.007,
    "size_thres": (25, None)}
param_second_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.4}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        param_save_minian = {
            "dpath": int_dir,
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        minian_ds = open_minian(int_dir)
        Y_fm_chk = minian_ds['Y_fm_chk']
        Y_hw_chk = minian_ds['Y_hw_chk']
        A = minian_ds['A']
        C = minian_ds['C']
        C_chk = minian_ds['C_chk']
        varr_ref = minian_ds['varr_ref']
        chk, _ = get_optimal_chk(varr_ref, dtype=float)

        A, C, C_chk, b, f, S, b0, c0 = run_cnmf(Y_hw_chk, 
                                         Y_fm_chk, 
                                         A, 
                                         C, 
                                         C_chk, 
                                         param_get_noise, 
                                         param_first_spatial, 
                                         param_first_temporal, 
                                         param_first_merge, 
                                         param_second_spatial, 
                                         param_second_temporal, 
                                         int_dir, 
                                         chk)
        
        A_path = os.path.join(int_dir, "A.csv")
        C_path = os.path.join(int_dir, "C.csv")
        S_path = os.path.join(int_dir, "S.csv")
        A.to_dataframe().reset_index().to_csv(A_path)
        C.to_dataframe().reset_index().to_csv(C_path)
        S.to_dataframe().reset_index().to_csv(S_path)

        A_path = os.path.join(int_dir, "A.h5")
        C_path = os.path.join(int_dir, "C.h5")
        S_path = os.path.join(int_dir, "S.h5")
        A.to_dataframe().reset_index().to_hdf(A_path, 'A')
        C.to_dataframe().reset_index().to_hdf(C_path, 'C')
        S.to_dataframe().reset_index().to_hdf(S_path, 'S')

In [ ]:
import pandas as pd

# Load the C.csv file
c_file_path = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-120/annateruel/social-discrimination-test/AD23-120/customEntValHere/2024_03_01/15_50_30/My_V4_Miniscope/C.csv'
c_data = pd.read_csv(c_file_path)

unique_unit_ids = c_data['unit_id'].unique()
num_unique_unit_ids = len(unique_unit_ids)
num_unique_unit_ids
